In [23]:
import pandas as pd
import numpy as np
import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/edwardlee/Desktop/chromedriver"

In [24]:
browser = webdriver.Chrome(executable_path = chromedriver)

In [25]:
# Get all years for 1996-2016
browser.get('http://stats.nba.com/league/player/')
HTML = browser.page_source
year_data = Selector(text=HTML).xpath("//select[@name='Season']/option/text()").extract()
browser.quit()

In [38]:
# Remove 2016-17 from list
year_data = year_data[1:]

In [27]:
trad_base_cols = [u'PLAYER_ID', u'PLAYER_NAME',
       u'TEAM_ABBREVIATION', u'AGE', u'GP', u'W', u'L', u'MIN',
       u'FGM', u'FGA', u'FG3M', u'FG3A', u'FTM', u'FTA',
       u'OREB', u'DREB', u'REB', u'AST', u'TOV', u'STL', u'BLK',
       u'BLKA', u'PF', u'PFD', u'PTS', u'PLUS_MINUS', u'DD2', u'TD3', 'SEASON']

In [28]:
trad_adv_cols = [u'PLAYER_ID', u'OFF_RATING', u'DEF_RATING', u'NET_RATING', u'AST_PCT', u'AST_TO',
       u'AST_RATIO', u'OREB_PCT', u'DREB_PCT', u'REB_PCT', u'TM_TOV_PCT',
       u'EFG_PCT', u'TS_PCT', u'USG_PCT', u'PACE', u'PIE', u'FGM_PG', 'SEASON']

In [29]:
trad_misc_cols = [u'PLAYER_ID', u'PTS_OFF_TOV', u'PTS_2ND_CHANCE', u'PTS_FB', u'PTS_PAINT',
       u'OPP_PTS_OFF_TOV', u'OPP_PTS_2ND_CHANCE', u'OPP_PTS_FB',
       u'OPP_PTS_PAINT', 'SEASON']

In [120]:
# trad_opp_cols = [u'VS_PLAYER_ID', u'OPP_FGM', u'OPP_FGA',
#         u'OPP_FG3M', u'OPP_FG3A', u'OPP_FTM',
#        u'OPP_FTA', u'OPP_OREB', u'OPP_DREB', u'OPP_REB',
#        u'OPP_AST', u'OPP_TOV', u'OPP_STL', u'OPP_BLK', u'OPP_BLKA', u'OPP_PF',
#        u'OPP_PFD', u'OPP_PTS', 'SEASON']

In [31]:
trad_score_cols = [u'PLAYER_ID', u'PCT_FGA_2PT', u'PCT_FGA_3PT', u'PCT_PTS_2PT', u'PCT_PTS_2PT_MR',
       u'PCT_PTS_3PT', u'PCT_PTS_FB', u'PCT_PTS_FT', u'PCT_PTS_OFF_TOV',
       u'PCT_PTS_PAINT', u'PCT_AST_2PM', u'PCT_UAST_2PM', u'PCT_AST_3PM',
       u'PCT_UAST_3PM', u'PCT_AST_FGM', u'PCT_UAST_FGM', 'SEASON']

In [32]:
trad_usage_cols = [u'PLAYER_ID', u'PCT_FGM', u'PCT_FGA', u'PCT_FG3M', u'PCT_FG3A',
       u'PCT_FTM', u'PCT_FTA', u'PCT_OREB', u'PCT_DREB', u'PCT_REB',
       u'PCT_AST', u'PCT_TOV', u'PCT_STL', u'PCT_BLK', u'PCT_BLKA', u'PCT_PF',
       u'PCT_PFD', u'PCT_PTS', 'SEASON']

In [33]:
shooting_cols = [u'PLAYER_ID', u'FGM_less_than_5_ft',
       u'FGA_less_than_5_ft', u'FG_PCT_less_than_5_ft', u'FGM_5-9_ft',
       u'FGA_5-9_ft', u'FG_PCT_5-9_ft', u'FGM_10-14_ft', u'FGA_10-14_ft',
       u'FG_PCT_10-14_ft', u'FGM_15-19_ft', u'FGA_15-19_ft',
       u'FG_PCT_15-19_ft', u'FGM_20-24_ft', u'FGA_20-24_ft',
       u'FG_PCT_20-24_ft', u'FGM_25-29_ft', u'FGA_25-29_ft',
       u'FG_PCT_25-29_ft', u'FGM_30-34_ft', u'FGA_30-34_ft',
       u'FG_PCT_30-34_ft', u'FGM_35-39_ft', u'FGA_35-39_ft',
       u'FG_PCT_35-39_ft', u'FGM_40+_ft', u'FGA_40+_ft', u'FG_PCT_40+_ft',
       u'SEASON']

In [34]:
opp_shooting_cols = ['PLAYER_ID', u'OPP_FGM_less_than_5_ft',
       u'OPP_FGA_less_than_5_ft', u'OPP_FG_PCT_less_than_5_ft',
       u'OPP_FGM_5-9_ft', u'OPP_FGA_5-9_ft', u'OPP_FG_PCT_5-9_ft',
       u'OPP_FGM_10-14_ft', u'OPP_FGA_10-14_ft', u'OPP_FG_PCT_10-14_ft',
       u'OPP_FGM_15-19_ft', u'OPP_FGA_15-19_ft', u'OPP_FG_PCT_15-19_ft',
       u'OPP_FGM_20-24_ft', u'OPP_FGA_20-24_ft', u'OPP_FG_PCT_20-24_ft',
       u'OPP_FGM_25-29_ft', u'OPP_FGA_25-29_ft', u'OPP_FG_PCT_25-29_ft',
       u'OPP_FGM_30-34_ft', u'OPP_FGA_30-34_ft', u'OPP_FG_PCT_30-34_ft',
       u'OPP_FGM_35-39_ft', u'OPP_FGA_35-39_ft', u'OPP_FG_PCT_35-39_ft',
       u'OPP_FGM_40+_ft', u'OPP_FGA_40+_ft', u'OPP_FG_PCT_40+_ft', 'SEASON']

In [139]:
### Merge per season/year by all stats/columns in that season
for year in year_data:
    base_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/traditional_stats/Base_%s.csv' % (year))
    adv_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/traditional_stats/Advanced_%s.csv' % (year))
    misc_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/traditional_stats/Misc_%s.csv' % (year))
    score_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/traditional_stats/Scoring_%s.csv' % (year))
    usage_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/traditional_stats/Usage_%s.csv' % (year))
#     opponent_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/traditional_stats/opponent_%s.csv' % (year))
    shooting_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/shooting_stats/Base_%s.csv' % (year))
    opp_shooting_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/shooting_stats/Opponent_%s.csv' % (year))
    
    # Add Season for each dataset season
    base_csv['SEASON'] = [year for i in range(base_csv.shape[0])]
    adv_csv['SEASON'] = [year for i in range(adv_csv.shape[0])]
    misc_csv['SEASON'] = [year for i in range(misc_csv.shape[0])]
    score_csv['SEASON'] = [year for i in range(score_csv.shape[0])]
    usage_csv['SEASON'] = [year for i in range(usage_csv.shape[0])]
#     opponent_csv['SEASON'] = [year for i in range(opponent_csv.shape[0])]
    shooting_csv['SEASON'] = [year for i in range(shooting_csv.shape[0])]
    opp_shooting_csv['SEASON'] = [year for i in range(opp_shooting_csv.shape[0])]
    
    # Keep columns that are not repeated and required
    base_csv         = base_csv[trad_base_cols]
    adv_csv          = adv_csv[trad_adv_cols]
    misc_csv         = misc_csv[trad_misc_cols]
#     opponent_csv     = opponent_csv[trad_opp_cols]
    score_csv        = score_csv[trad_score_cols]
    usage_csv        = usage_csv[trad_usage_cols]
    shooting_csv     = shooting_csv[shooting_cols]
    opp_shooting_csv = opp_shooting_csv[opp_shooting_cols]
    
    # Group by same players playing on different teams per season
#     opponet_csv = opponent_csv.groupby(['VS_PLAYER_ID', 'SEASON']).sum().reset_index()
    
    # Change column name for merging
#     opponent_csv.rename(columns={'VS_PLAYER_ID':'PLAYER_ID'}, inplace=True)
    
    # Merge all datasets together per Season
    df_merge = base_csv.merge(adv_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                       .merge(misc_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                       .merge(score_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                       .merge(usage_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                        .merge(shooting_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                        .merge(opp_shooting_csv, on=['PLAYER_ID', 'SEASON'], how='outer')
#                        .merge(opponent_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
    
    # Save each merged season into different datasets
    link = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/merged/trad_shooting_merged_%s.csv' % year
    df_merge.to_csv(link)

In [140]:
df_merge.columns.tolist()

['PLAYER_ID',
 'PLAYER_NAME',
 'TEAM_ABBREVIATION',
 'AGE',
 'GP',
 'W',
 'L',
 'MIN',
 'FGM',
 'FGA',
 'FG3M',
 'FG3A',
 'FTM',
 'FTA',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'DD2',
 'TD3',
 'SEASON',
 'OFF_RATING',
 'DEF_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TO',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'PACE',
 'PIE',
 'FGM_PG',
 'PTS_OFF_TOV',
 'PTS_2ND_CHANCE',
 'PTS_FB',
 'PTS_PAINT',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_PAINT',
 'PCT_FGA_2PT',
 'PCT_FGA_3PT',
 'PCT_PTS_2PT',
 'PCT_PTS_2PT_MR',
 'PCT_PTS_3PT',
 'PCT_PTS_FB',
 'PCT_PTS_FT',
 'PCT_PTS_OFF_TOV',
 'PCT_PTS_PAINT',
 'PCT_AST_2PM',
 'PCT_UAST_2PM',
 'PCT_AST_3PM',
 'PCT_UAST_3PM',
 'PCT_AST_FGM',
 'PCT_UAST_FGM',
 'PCT_FGM',
 'PCT_FGA',
 'PCT_FG3M',
 'PCT_FG3A',
 'PCT_FTM',
 'PCT_FTA',
 'PCT_OREB',
 'PCT_DREB',
 'PCT_REB',
 'PCT_AST',
 'PCT_TOV',
 'PCT_STL',

In [158]:
# Columns that are used to rank
ranked_cols = ['GP',
 'W',
 'L',
 'MIN',
 'FGM',
 'FG3M',
 'FTM',
 'OREB',
 'DREB',
 'REB',
 'AST',
 'TOV',
 'STL',
 'BLK',
 'BLKA',
 'PF',
 'PFD',
 'PTS',
 'PLUS_MINUS',
 'DD2',
 'TD3',
 'SEASON',
 'OFF_RATING',
 'DEF_RATING',
 'NET_RATING',
 'AST_PCT',
 'AST_TO',
 'AST_RATIO',
 'OREB_PCT',
 'DREB_PCT',
 'REB_PCT',
 'TM_TOV_PCT',
 'EFG_PCT',
 'TS_PCT',
 'USG_PCT',
 'PACE',
 'PIE',
 'FGM_PG',
 'PTS_OFF_TOV',
 'PTS_2ND_CHANCE',
 'PTS_FB',
 'PTS_PAINT',
 'OPP_PTS_OFF_TOV',
 'OPP_PTS_2ND_CHANCE',
 'OPP_PTS_FB',
 'OPP_PTS_PAINT',
 'PCT_FGA_2PT',
 'PCT_FGA_3PT',
 'PCT_PTS_2PT',
 'PCT_PTS_2PT_MR',
 'PCT_PTS_3PT',
 'PCT_PTS_FB',
 'PCT_PTS_FT',
 'PCT_PTS_OFF_TOV',
 'PCT_PTS_PAINT',
 'PCT_AST_2PM',
 'PCT_UAST_2PM',
 'PCT_AST_3PM',
 'PCT_UAST_3PM',
 'PCT_AST_FGM',
 'PCT_UAST_FGM',
 'PCT_FGM',
 'PCT_FGA',
 'PCT_FG3M',
 'PCT_FG3A',
 'PCT_FTM',
 'PCT_FTA',
 'PCT_OREB',
 'PCT_DREB',
 'PCT_REB',
 'PCT_AST',
 'PCT_TOV',
 'PCT_STL',
 'PCT_BLK',
 'PCT_BLKA',
 'PCT_PF',
 'PCT_PFD',
 'PCT_PTS',
 'FGM_less_than_5_ft',
 'FG_PCT_less_than_5_ft',
 'FGM_5-9_ft',
 'FG_PCT_5-9_ft',
 'FGM_10-14_ft',
 'FG_PCT_10-14_ft',
 'FGM_15-19_ft',
 'FG_PCT_15-19_ft',
 'FGM_20-24_ft',
 'FG_PCT_20-24_ft',
 'FGM_25-29_ft',
 'FG_PCT_25-29_ft',
 'OPP_FGM_less_than_5_ft',
 'OPP_FG_PCT_less_than_5_ft',
 'OPP_FGM_5-9_ft',
 'OPP_FG_PCT_5-9_ft',
 'OPP_FGM_10-14_ft',
 'OPP_FG_PCT_10-14_ft',
 'OPP_FGM_15-19_ft',
 'OPP_FG_PCT_15-19_ft',
 'OPP_FGM_20-24_ft',
 'OPP_FG_PCT_20-24_ft',
 'OPP_FGM_25-29_ft',
 'OPP_FG_PCT_25-29_ft']

In [174]:
### Sort values by columns and then rank each column, then agg the ranked for each player per season
### to get an average rank.

for year in year_data:
    csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/merged/trad_shooting_merged_%s.csv' % year
    df = pd.read_csv(csv)
    
    # Through the added columns, sort each column then rank each player from 1 to length of dataset
    for col in ranked_cols:
        if 'OPP' in col:
            if ('PF' in col) or ('TOV' in col):
                df.sort_values(col, ascending=False, na_position='last', inplace=True)
                new_col_name = 'rank_%s' % col
                df[new_col_name] = [i+1 for i in range(df.shape[0])]
            else:
                df.sort_values(col, ascending=True, na_position='last', inplace=True)
                new_col_name = 'rank_%s' % col
                df[new_col_name] = [i+1 for i in range(df.shape[0])]
        elif ('L' in col) or ('PF' in col) or ('TOV' in col) or ('Miss' in col):
            df.sort_values(col, ascending=True, na_position='last', inplace=True)
            new_col_name = 'rank_%s' % col
            df[new_col_name] = [i+1 for i in range(df.shape[0])]
        else:
            df.sort_values(col, ascending=False, na_position='last', inplace=True)
            new_col_name = 'rank_%s' % col
            df[new_col_name] = [i+1 for i in range(df.shape[0])]
    
    df['agg_rank'] = df.ix[:,'rank_GP':].mean(axis=1)
    
    link = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/tradshoot_merged_%s_withRANK.csv' % year
    df.to_csv(link)

In [67]:
# Percentage Missed Games for game_risk
print '10% or less missed games: ', 82,'-',np.ceil(82 * .9)
print '10% - 25% missed games: ', np.ceil(82 * .9), '-', np.ceil(82 * .75)
print '25% - 50% missed games: ', np.ceil(82 * .75), '-', np.ceil(82 * .50)
print '50% or more missed games: ', np.ceil(82 * .50), '-', 0

10% or less missed games:  82 - 74.0
10% - 25% missed games:  74.0 - 62.0
25% - 50% missed games:  62.0 - 41.0
50% or more missed games:  41.0 - 0


In [72]:
# Percentage Missed Games for game_risk in 1998-99 lockout (50 games played)
print '10% or less missed games: ', 50,'-',np.ceil(50 * .9)
print '10% - 25% missed games: ', np.ceil(50 * .9), '-', np.ceil(50 * .75)
print '25% - 50% missed games: ', np.ceil(50 * .75), '-', np.ceil(50 * .50)
print '50% or more missed games: ', np.ceil(50 * .50), '-', 0

10% or less missed games:  50 - 45.0
10% - 25% missed games:  45.0 - 38.0
25% - 50% missed games:  38.0 - 25.0
50% or more missed games:  25.0 - 0


In [73]:
# Percentage Missed Games for game_risk in 2011-12 lockout (66 games played)
print '10% or less missed games: ', 66,'-',np.ceil(66 * .9)
print '10% - 25% missed games: ', np.ceil(66 * .9), '-', np.ceil(66 * .75)
print '25% - 50% missed games: ', np.ceil(66 * .75), '-', np.ceil(66 * .50)
print '50% or more missed games: ', np.ceil(66 * .50), '-', 0

10% or less missed games:  66 - 60.0
10% - 25% missed games:  60.0 - 50.0
25% - 50% missed games:  50.0 - 33.0
50% or more missed games:  33.0 - 0


In [175]:
### MERGE TRAD AND SHOOTING STATS BY CATEGORIES FROM 1996-97 to 2015-16

trad_rank = []

for year in year_data:
    csv_file = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/tradshoot_merged_%s_withRANK.csv' % year
    df_tmp = pd.read_csv(csv_file)
    df_tmp.drop('Unnamed: 0', axis=1, inplace=True)
    
    game_risk = []
    # Add new column game_risk and lock out periods included
    for games in df_tmp['GP']:
        if year == '1998-99':
            if games >= 45:
                game_risk.append(0)
            elif games < 45 and games >= 38:
                game_risk.append(1)
            elif games < 38 and games >= 25:
                game_risk.append(2)
            elif games < 25:
                game_risk.append(3)
        elif year == '2011-12':
            if games >= 60:
                game_risk.append(0)
            elif games < 60 and games >= 50:
                game_risk.append(1)
            elif games < 50 and games >= 33:
                game_risk.append(2)
            elif games < 33:
                game_risk.append(3)
        else:
            if games >= 74:
                game_risk.append(0)
            elif games < 74 and games >= 62:
                game_risk.append(1)
            elif games < 62 and games >= 41:
                game_risk.append(2)
            elif games < 41:
                game_risk.append(3)
    
    df_tmp['game_risk'] = game_risk
    trad_rank.append(df_tmp)

trad_rank_df = pd.concat(trad_rank)
trad_rank_df.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/completed/trad_rank_target.csv')

In [176]:
trad_rank_df.shape

(9077, 244)

In [177]:
trad_rank_df['OPP_FGM_25-29_ft'].isnull().sum()

44

In [185]:
trad_rank_df.sort_values('agg_rank', ascending=True).reset_index(drop=True).drop("Unnamed: 0.1", axis=1).head(25)

,PLAYER_ID,PLAYER_NAME,TEAM_ABBREVIATION,AGE,GP,W,L,MIN,FGM,FGA,...,rank_OPP_FGM_10-14_ft,rank_OPP_FG_PCT_10-14_ft,rank_OPP_FGM_15-19_ft,rank_OPP_FG_PCT_15-19_ft,rank_OPP_FGM_20-24_ft,rank_OPP_FG_PCT_20-24_ft,rank_OPP_FGM_25-29_ft,rank_OPP_FG_PCT_25-29_ft,agg_rank,game_risk
0,708,Kevin Garnett,MIN,27.0,82,51,31,3321.788333,743,1481,...,362,86,427,271,423,165,399,91,153.833333,0
1,708,Kevin Garnett,MIN,28.0,82,58,24,3235.055000,804,1611,...,438,244,434,75,426,208,365,74,155.647059,0
2,1717,Dirk Nowitzki,DAL,25.0,80,59,21,3121.160000,690,1489,...,393,212,399,313,425,179,333,99,157.598039,0
3,1717,Dirk Nowitzki,DAL,23.0,82,53,29,3122.753333,591,1247,...,422,146,385,192,432,136,301,46,160.578431,0
4,1495,Tim Duncan,SAS,27.0,81,60,21,3182.445000,714,1392,...,422,229,411,230,386,178,324,113,162.774510,0
5,978,Peja Stojakovic,SAC,26.0,72,51,21,2451.356667,497,1034,...,336,227,387,207,392,115,321,118,163.166667,1
6,1717,Dirk Nowitzki,DAL,24.0,76,53,23,2891.015000,600,1258,...,403,352,348,142,433,203,298,109,163.392157,0
7,1503,Tracy McGrady,ORL,24.0,75,39,36,2949.775000,829,1813,...,397,299,405,177,384,259,368,117,163.558824,0
8,2544,LeBron James,CLE,24.0,81,66,15,3053.826667,789,1613,...,405,114,402,73,400,96,359,47,164.372549,0
9,708,Kevin Garnett,MIN,24.0,81,50,31,3243.250000,759,1526,...,374,232,422,105,411,236,412,196,164.980392,0
